# _emcee_ et convergence

Au dernier TP, nous avons implémenté un MCMC Metropolis-Hastings nous-mêmes. En pratique, il est généralement plus simple et plus robuste d'utiliser une librairie bien testée et qui permet d'explorer différentes distributions de proposition. Pour le reste du cours, nous ferons les MCMCs avec [_emcee_](https://emcee.readthedocs.io/en/stable/). Par défaut, _emcee_ utilise le _stretch move_ que nous avons vu en classe pour proposer des échantillons.

En plus d'implémenter un échantillonneur (_sampler_), _emcee_ inclut également des fonctions pour vérifier la convergence de nos chaînes. D'ailleurs, les diapositives sur l'autocorrélation sont inspirées par le tutoriel  [Autocorrelation analysis & convergence](https://emcee.readthedocs.io/en/stable/tutorials/autocorr/) d'_emcee_.

Dans ce TP, nous allons apprendre à utiliser _emcee voir en pratique comment certains facteurs affectent la convergence. Nous utiliserons la fonction `get_autocorr_time()` d'_emcee_ pour calculer le temps d'autocorrélation de nos chaînes. Cette fonction nous avertira également si nos chaînes sont moins longues que 50 fois la valeur estimée du temps d'autocorrélation.


Plan:
1. [MCMC appliqué à l'analyse de données](#section-1)
1. [Distributions de proposition et convergence](#section-2)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import corner
import emcee

In [ ]:
rcParams["font.size"] = 18
rcParams["figure.figsize"] = (9, 6)
rcParams["xtick.direction"] = "in"
rcParams["ytick.right"] = True

<a id="section-1"></a>
## 1 - MCMC appliqué à l'analyse de données

Dans cette section, nous allons appliquer _emcee_ pour modéliser un signal périodique.

### Données synthétiques et vraisemblance maximale

Les première étapes consistent à générer des données synthétiques autour d'une droite, à estimer les paramètres ayant une vraisemblance (sans les _priors_) maximale et à les utiliser comme point de départ dans un MCMC qui échantillone la distribution à posteriori.

In [ ]:
# Cette cellulle est tirée du tutoriel emcee pour obtenir exactement les mêmes données
np.random.seed(123)

# Choose the "true" parameters.
m_true = -0.9594
b_true = 4.294
f_true = 0.534

# Generate some synthetic data from the model.
N = 50
x = np.sort(10 * np.random.rand(N))
yerr = 0.1 + 0.5 * np.random.rand(N)
y = m_true * x + b_true
y += np.abs(f_true * y) * np.random.randn(N)
y += yerr * np.random.randn(N)

plt.errorbar(x, y, yerr=yerr, fmt=".k", capsize=0, label="Données simulées")
x0 = np.linspace(0, 10, 500)
plt.plot(x0, m_true * x0 + b_true, "k", alpha=0.3, lw=3, label="Vrai signal")
plt.legend()
plt.xlim(0, 10)
plt.xlabel("x")
plt.ylabel("y")
plt.show()

Nous aurons besoin de la fonction de vraisemblance pour le premier estimé. Elle pourra être réutilisée plus tard pour définir la distribution à posteriori.

Notre modèle physique est donné par une droite

$$
f(x) = m x + b.
$$

Nous utiliserons une vraisemblance gaussienne, mais ajouterons un paramètre additionnel au cas où les erreurs aient été sous-estimées, de sorte que la variance totale est donnée par:

$$
s_n^2 = \sigma_n^2 + (m x_n + b)^2 f^2
$$

où $\sigma_n^2$ correspond à `yerr` pour chaque point. Il est généralement suggéré d'utiliser un tel facteur ou, encore plus simple, un paramètre $sigma^2$ tel que $s_n^2 = \sigma_n^2 + \sigma^2$ pour s'assurer que les barres d'erreur ne sont pas sous-estimées. Si elles ne l'étaient pas, le paramètre convergera simplement vers 0.

**Exercice: Implémentez la fonction de log-vraisemblance**

In [ ]:
def log_likelihood(theta, x, y, yerr):
    """
    - theta: vecteur de paramètres m, b et log_f
    - x, y, yerr: données
    """
    pass

In [ ]:
from scipy.optimize import minimize

np.random.seed(42)
nll = lambda *args: -log_likelihood(*args)
initial = np.array([m_true, b_true, np.log(f_true)]) + 0.1 * np.random.randn(3)
soln = minimize(nll, initial, args=(x, y, yerr))
m_ml, b_ml, log_f_ml = soln.x

print("Maximum likelihood estimates:")
print("m = {0:.3f}".format(m_ml))
print("b = {0:.3f}".format(b_ml))
print("f = {0:.3f}".format(np.exp(log_f_ml)))

plt.errorbar(x, y, yerr=yerr, fmt=".k", capsize=0, label="Données")
plt.plot(x0, m_true * x0 + b_true, "k", alpha=0.3, lw=3, label="Vrai signal")
plt.plot(x0, np.dot(np.vander(x0, 2), [m_ml, b_ml]), ":k", label="Estimé vraisemblance maximale")
plt.legend(fontsize=14)
plt.xlim(0, 10)
plt.xlabel("x")
plt.ylabel("y")
plt.show()

On peut voir que le modèle de vraisemblance maximale est déjà en bon accord avec le la vraie droite. Par contre, il n'y a pas de barres d'erreurs sur nos paramètres et nous n'avons specifiés aucuns _priors_.

### Exemple initial de MCMC

On peut d'abord reproduire l'exemple _emcee_ avec les mêmes _priors_ pour s'assurer que tout fonctionne comme prévu.
On implémente tout en espace log, car c'est ce qu'utilise _emcee_, pour les raison numériques dont nous avons discuté en classe.

Remarquez qu'on ne calcule pas le _likelihood_ si le _prior_ est 0 (log-prior $ = -\infty$), pour sauver du temps de calcul.

**Exercice: Implémentez la fonction log-prior et log-posterior**
**Utilisez un prior entre -5 et 5 pour m, 0 et 10 pour b et -10 et 1 pour log(f)**

In [ ]:
def log_prior(theta):
    pass

def log_probability(theta, x, y, yerr):
    pass

On peut maintenant échantilloner avec _emcee_. Quelques points importants:

- _emcee_ échantillonne en parallèle avec plusieurs chaînes (32 ici).
- On doit donc initialiser des "marcheurs" dans un tableau avec 32 positions aléatoires pour chacun de nos 3 paramètres.
- Pour l'initialisation, généralement, une petite "balle" gaussienne suffit. Les marcheurs vont s'étendre à l'espace des paramètres assez rapidement.
- Il y a une phase de _burn in_ ou _warm-up_ au début qui permet aux marcheurs de trouver l'ensemble typique du _posterior_.

**Exercice: implémentez un MCMC avec emcee et exécutez le pour 5000 pas. Démarrez au point de vraisemblance maximale. Affichez ensuite l'évolution des chaînes, un _corner_ plot et calculez le temps d'autocorrélation.**

**Exercice: calculez les incertitudes données par corner manuellement pour les vérifier**

On peut voir que $m$ et $b$ sont fortement anti-corrélés. Un changement dans l'un affectera fortement l'autre dans le MCMC et dans la solution finale

Pour bien vérifier que le modèle correspond aux données, on peut également afficher des échantillons du MCMC superposés aux données.

**Exercice: affichez le modèle pour 100 échantillons de votre MCMC avec les données et le vrai signal**

### Effet de la longueur des chaines

Si on répète le processus, mais qu'on s'arrête avec des chaînes trop courtes, l'échantillonage sera incomplet. _emcee_ nous le laissera savoir lorsqu'on calcule le temps d'autocorrélation, mais l'inspection visuelle nous aidera aussi.

**Exercice: Répétez l'analyse ci-dessus mais avec des chaînes de 500 pas seulement.**

Ici, on peut utiliser l'option `quiet=True` pour avoir un avertissemment au lieu d'une erreur. Le fait que l'on puisse distinguer des variations claires dans les chaînes sur une échelle inférieure à ~ 1/50 de notre chaîne est un autre indicateur (quoique beaucoup moins rigoureux).

Dans ce cas-ci, nous aurions donc eu besoin de plus de pas pour atteindre la convergence (mais le MCMC semblait sur la bonne voie).

### Effet de la distribution à priori

La distribution à priori (le _prior_) peut aussi avoir un impact sur la convergence.

#### Prior trop étroit

Un prior trop étroit sur un ou plusieurs paramètres peut restreindre les valeurs explorées et nous mener à sous-estimer les incertitudes. Dans un tel cas, il n'y a pas un problème de convergence à proprement parler, car la distribution peut être bien échantillonée quand même. Le problème est plutôt dans le _choix de distribution_.

**Exercice: Utilisez un _prior_ sur $m$ est entre -1.1 et -0.9 et répétez les diagnotics et analyses ci-dessus**

L'effet décrit plus haut est aussi observable ici. Les bords de la distribution en $m$ sont coupés par le prior. A moins d'avoir une bonne raison de forcer le paramètre dans cette intervalle (par exemple une contrainte physique), il serait préférable d'élargir le prior.

On peut aussi remarquer que le paramètre $b$ a lui aussi une forme plus coupée, en raison de la corrélation entre $m$ et $b$ (en limitant $m$, on limite $b$ du même coup).

#### Prior qui exclue la solution

En définissant un prior qui exclue la solution, nous forcerons $m$ vers des valeurs qui ne représentent pas la meilleure solution.

**Exercice: Testez un prior qui exclue la solution. Quel est l'impact sur la distribution à posterior?**

Avec les deux graphiques ci-dessus, on voit encore que la pente n'est pas du tout en accord avec les données. Il serait préférable de reconsidérer notre _prior_ pour s'assurer qu'il a du sens physiquement et qu'il permet plus de valeurs en $m$.

### Effet de la position de départ des marcheurs

En changeant la position de départ des marcheurs, la forme des chaînes peut changer. Surtout pendant le _burn-in_. S'il y a des bi-modalités, elles pourraient également être révélées. En général, c'est une bonne idée de répéter le MCMC avec  différentes valeurs initiales pour voir s'il y a toujours une convergence vers la même forme de distribution.

#### Dans le prior, mais plus loin du mode

Dans les sections précédentes, notre initialisation était très près du mode de la distribution. On peut tester des valeurs volontairement plus éloignées pour voir si on retrouve la même solution.

**Exercice: Testez des valeurs initiales différentes pour les paramètres (exemple `[-1.5, 2.0, -1.0]`)**

#### Valeur initiale hors du prior

Il peut arriver, par inattention, qu'on initialise les marcheurs hors du _prior_. C'est habituellement assez facile à identifier.

Quelques exemples de solutions possibles:

- Changer manuellement les valeurs initiales.
- Optimiser le maximum à posteriori (MAP) au lieu de la vraisemblance maximale (ainsi notre valeur initiale tient compte des priors)
- Réviser nos priors, l'erreur pourrait venir d'une borne mal définie et non de la position initiale des marcheurs.
- Ajouter une vérification pour s'assurer que nos valeurs initiales donnent une valeur de _prior_ finie avant de commencer le MCMC (ça ne règle pas le problème, mais ça le détecte plus clairement).

Voici un exemple de macheurs qui démarrent hors du _prior_.

Les chaînes sont droites car le _prior_ (et le _posterior_ ensuite) retourne $-\infty$. La probabilité ne change donc jamais et tous les pas sont refusés. Dans un tel cas, l'autocorrélation sera indéfiniie (`nan`). On voit aussi que le _corner plot_ n'affiche pas de distribution claire, mais plus tôt les positions initiales des marcheurs.

<a id="section-2"></a>
## 2 - Distributions de proposition et convergence

Dans la prochaine section, nous explorerons des distributions simples pour voir l'effet de la distribution de proposition (_moves_ dans _emcee_) sur la convergence.

Commençons avec une distribution bimodale en 1D (cette distribution est la même que dans le tutoriel "[Using different moves](https://emcee.readthedocs.io/en/stable/tutorials/moves/)" d'_emcee_). Ici, au lieu d'échantillonner une distribution postérieure pour déterminer les paramètres d'un modèle, nous allons seulement tirer des échantillons d'une distribution connue afin de comparer la convergence de différentes méthodes.

Nous utiliserons trois distributions de proposition différente:
1. Une distribution Gaussienne avec le Metropolis-Hastings (`emcee.moves.Gaussian()`)
2. "Stretch move" (`emcee.moves.Stretch()`, la distribution par défaut d'_emcee_)
3. MCMC par évolution différentielle (`emcee.moves.DEMove()` et `emcee.moves.DESnookerMove()`)

Ces distributions sont déjà implémentées dans _emcee_ et peuvent être spécifiées au moment de créer notre "_sampler_" (objet `emcee.EnsembleSampler`).

Nous pouvons d'abord définir notre distribution et, comme c'est une distribution facile à calculer analytiquement, nous pouvons l'afficher directement (ce n'est habituellement pas possible avec les distributions à posteriori que nous utilisons en physique ou en analyse de données).

In [ ]:
def logprob_bimod(x):
    return np.sum(
        np.logaddexp(-0.5 * (x - 2) ** 2, -0.5 * (x + 2) ** 2)
        - 0.5 * np.log(2 * np.pi)
        - np.log(2)
    )

In [ ]:
x = np.linspace(-6, 6, num=1000)
y = np.exp(list(map(logprob_bimod, x)))

In [ ]:
plt.plot(x, y, "k")
plt.yticks([])
plt.xlim(-5.5, 5.5)
plt.ylabel("p(x)")
plt.xlabel("x")
plt.show()

### Metropolis-Hastings

Nous allons d'abord utiliser l'algorithme de Metropolis-Hastings avec une distribution de proposition gaussienne. 

Remarquez que le paramètre `cov` (qui n'est qu'une variance dans ce cas-ci, en 1D) peut être ajusté. Une distribution bimodale est un exemple de cas où il peut être avantageux d'utiliser une covariance un peu plus élevée. Avoir une échelle plus grande permet de plus efficacement se déplacer entre les modes de la distribution. Vous pouvez vous amuser à varier ce paramètres et voir l'effet sur le temps d'autocorrélation.

In [ ]:
init = np.random.randn(32, 1)
nwalkers, ndim = init.shape

sampler = emcee.EnsembleSampler(
    nwalkers, ndim, logprob_bimod, moves=emcee.moves.GaussianMove(cov=1.0)
)
sampler.run_mcmc(init, 5000, progress=True)

fchain_gauss = sampler.get_chain(flat=True)

print(f"Temps d'autocorrélation intégré, Metropolis-Hastings: {sampler.get_autocorr_time()}")

On peut ensuite afficher un seul marcheur pour voir les détails de l'échantillonage.

In [ ]:
plt.plot(sampler.get_chain()[:, 0, 0], "k", lw=0.5)
plt.xlim(0, 5000)
plt.ylim(-5.5, 5.5)
plt.title("move: GaussianMove", fontsize=20)
plt.xlabel("step number")
plt.ylabel("x")
plt.show()

### "_Stretch Move_" par défaut d'_emcee_

Maintenant, nous pouvons tester le "_stretch move_" d'_emcee_, qui est utilisé par défaut. Il utilise la position des autre marcheurs pour orienter les pas, donc certaines distributions (avec des covariance, par exemple) peuvent être mieux échantillonées.

Comme pour la variance du Metropolis-Hastings, l'échelle `a` pour le "Stretch" peut être ajustée. La valeur par défaut est `a=2.0`. Ici, une valeur de `3.0` semble améliorer l'efficacité (temps d'autocorrélation plus faible).

In [ ]:
init = np.random.randn(32, 1)
nwalkers, ndim = init.shape

sampler = emcee.EnsembleSampler(
    nwalkers, ndim, logprob_bimod, moves=emcee.moves.StretchMove(a=3.0)
)
sampler.run_mcmc(init, 5000, progress=True);

fchain_stretch = sampler.get_chain(flat=True)

print(f"Temps d'autocorrélation intégré, Stretch: {sampler.get_autocorr_time()}")

In [ ]:
plt.plot(sampler.get_chain()[:, 0, 0], "k", lw=0.5)
plt.xlim(0, 5000)
plt.ylim(-5.5, 5.5)
plt.title("move: StretchMove", fontsize=20)
plt.xlabel("step number")
plt.ylabel("x")
plt.show()

On peut voir ci-dessus que le temps d'autocorrélation est plus court que pour le Metropolis. Donc l'échantillonage est plus efficace (il faudra moins de pas pour atteindre $N \geq \tau_f$).

### MCMC par évolution différentielle

Certaines méthodes pour proposer de nouveaux pas dans _emcee_ utilisent l'évolution différentielle. Je laisse un lien vers la page [Moves](https://emcee.readthedocs.io/en/stable/user/moves/) de la documentation pour les intéressé·e·s. Dans les cas légèrement bimodaux comme celui-ci. le MCMC par évolution différentielle peut être plus efficace.

Il y a deux type de propositions par évolution différentielle (`DEMove()` et `DESnookerMove()`), donc nous allons combiner les deux en tirant 80 % des pas avec le premier et 20 % des pas avec le deuxième.

In [ ]:
init = np.random.randn(32, 1)
nwalkers, ndim = init.shape

sampler = emcee.EnsembleSampler(nwalkers, ndim, logprob_bimod, moves=[
        (emcee.moves.DEMove(), 0.8),
        (emcee.moves.DESnookerMove(), 0.2),
    ])
sampler.run_mcmc(init, 5000, progress=True)

fchain_de = sampler.get_chain(flat=True)

print(f"Temps d'autocorrélation intégré, DE-MCMC: {sampler.get_autocorr_time()}")

In [ ]:
plt.plot(sampler.get_chain()[:, 0, 0], "k", lw=0.5)
plt.xlim(0, 5000)
plt.ylim(-5.5, 5.5)
plt.title("move: DEMove et DESnookerMove", fontsize=20)
plt.xlabel("step number")
plt.ylabel("x")
plt.show()

On voit que le temps d'autocorrélation est beaucoup plus petit que pour les deux premières distributions de proposition. Même à l'oeil, on voit que le marcheur explore plus rapidement.

En général. la configuration par défaut d'_emcee_ (i.e. sans spécifier de `move` au `EnsembleSampler()`) fonctionne très bien. Par contre, si le temps de convergence est très long. Il peut être pertinent de varier les paramètres et le type de pas pour voir l'effet sur la performance du MCMC.

Dans ce cas-ci, on peut voir que les 3 méthodes représentent bien la distribution analytique, malgré leurs différents temps de convergence. Ce ne sera pas toujours le cas avec les distributions à plusieurs dimensions auxquelles nous faisons généralement face en analyse de données.

In [ ]:
plt.hist(fchain_gauss, bins=50, density=True, histtype="step", label="M-H gaussien")
plt.hist(fchain_stretch, bins=50, density=True, histtype="step", label="Stretch")
plt.hist(fchain_de, bins=50, density=True, histtype="step", label="DE-MCMC")
plt.plot(x, y, "k", label="Courbe analytique")
plt.yticks([])
plt.ylabel("p(x)")
plt.xlabel("x")
plt.legend()
plt.show()